# JulES as a medium-term prognosis model

### Import packages

In [ ]:
using Pkg; Pkg.status()
# Pkg.add("Revise"); Pkg.add("Plots"); Pkg.add("PlotlyJS"); Pkg.add("PrettyTables")
# Pkg.update("TuLiPa") # uncomment to update TuLiPa to latest version
# Pkg.develop(path=joinpath(dirname(dirname(pwd())),"TuLiPa")); Pkg.status() # go to development version
# Pkg.undo(); Pkg.status() # go back to main package version
# Pkg.add(url="https://github.com/NVE/TuLiPa.git"); Pkg.status() # alternative go back to latest version

In [ ]:
using DataFrames, Statistics, JSON, Distributed, Clustering, YAML, Distributions, Revise, Plots, PrettyTables
plotlyjs(); # uncomment for interactive plots

In [ ]:
# config = YAML.load_file(joinpath("data", "config_jules_prognose.yml")) # config without datasets
config = YAML.load_file(joinpath(dirname(dirname(pwd())), "JulESIO", "config_jules_prognose_demo.yml")) # config with NVE datasets
weatheryear = config["main"]["weatheryears"][1]
datayear = config["main"]["datayears"][1]

### Prepare parallell processing - import code on all cores

In [ ]:
const numcores = config["main"]["numcores"]

if nprocs() < numcores
    addprocs(numcores - nprocs())
end

@show nprocs();

In [ ]:
@everywhere using TuLiPa, Dates
# @everywhere include(joinpath(dirname(dirname(pwd())),"TuLiPa/src/TuLiPa.jl"));

In [ ]:
@everywhere using JulES

In [ ]:
function getdataset(config, weatheryear)
    iprogtype = get(config["main"], "iprogtype", "direct")
    useifm = iprogtype != "direct"

    settings = config[config["main"]["settings"]]

    sti_dataset = joinpath(config["main"]["inputpath"], "static_input")
    weekstart = config["main"]["weekstart"]

    sti_dataset1 = joinpath(config["main"]["inputpath"], "Uke_$weekstart", "input")

    exd = JSON.parsefile(joinpath(sti_dataset1, "exogenprices_prognose1.json"))
    exogen = JulES.TuLiPa.getelements(exd, sti_dataset1)

    add = JSON.parsefile(joinpath(sti_dataset, "aggdetd2.json"))
    aggdetd = JulES.TuLiPa.getelements(add, sti_dataset)

    ipad = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoseragg$weatheryear.json"))
    agginflow = JulES.TuLiPa.getelements(ipad, sti_dataset1)

    thd = JSON.parsefile(joinpath(sti_dataset, "termisk1.json"))
    thermal = JulES.TuLiPa.getelements(thd, sti_dataset)

    wsd = JSON.parsefile(joinpath(sti_dataset, "vindsol.json"))
    windsol = JulES.TuLiPa.getelements(wsd, sti_dataset)

    trd = JSON.parsefile(joinpath(sti_dataset1, "nett.json"))
    transm = JulES.TuLiPa.getelements(trd)

    cod = JSON.parsefile(joinpath(sti_dataset, "forbruk5.json"))
    cons = JulES.TuLiPa.getelements(cod, sti_dataset)

    fpd = JSON.parsefile(joinpath(sti_dataset1, "brenselspriser.json"))
    fuel = JulES.TuLiPa.getelements(fpd, sti_dataset1)

    nud = JSON.parsefile(joinpath(sti_dataset1, "nuclear.json"))
    nuclear = JulES.TuLiPa.getelements(nud, sti_dataset1)

    dse = JSON.parsefile(joinpath(sti_dataset, "tidsserier_detd.json"))
    detdseries = JulES.TuLiPa.getelements(dse, sti_dataset)

    dda = JSON.parsefile(joinpath(sti_dataset, "dataset_detd.json"))
    detdstructure = JulES.TuLiPa.getelements(dda)

    ipd = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoser$weatheryear.json"))
    inflow = JulES.TuLiPa.getelements(ipd, sti_dataset1)

    progelements = vcat(exogen, aggdetd, thermal, windsol, transm, cons, agginflow, fuel, nuclear)
    aggstartmagdict = JSON.parsefile(joinpath(sti_dataset1, "aggstartmagdict.json"), dicttype=Dict{String, Float64})

    # --- inflow model (ifm) ----
    ifm_replacemap = Dict{String, String}()
    ifm_names = String[]
    ifm_weights = Dict{String, Dict{String, Float64}}()
    ifm_normfactors = Dict{String, Float64}()
    ifm_elements = JulES.TuLiPa.DataElement[]
    if useifm
        ifm_weights = JSON.parsefile(joinpath(sti_dataset1, "ifm_weights.json"))
        ifm_weights = Dict{String, Dict{String, Float64}}(ifm_weights)
        ifm_normfactors = JSON.parsefile(joinpath(sti_dataset, "ifm_normfactors.json"), dicttype=Dict{String, Float64})
        # TODO: Remove ifm_stationname_to_stationid if unused
        ifm_stationname_to_stationid = JSON.parsefile(joinpath(sti_dataset1, "ifm_stationname_to_stationid.json"), dicttype=Dict{String, String})
        ifm_empscode_to_stationid = JSON.parsefile(joinpath(sti_dataset, "ifm_empscode_to_station.json"), dicttype=Dict{String, String})
        for e in aggdetd
            if e.typename == "PrognosisSeriesParam"
                inflow_name = join(split(e.instancename, "_")[2:end], "_")
                ifm_replacemap[e.instancename] = inflow_name
            end
        end
        for e in detdstructure
            # NB! Code below relies on naming convension for Profile in PrognosisSeriesParam
            if e.typename == "PrognosisSeriesParam"
                empscode = join(split(e.value["Profile"], "_")[2:end], "_")
                if haskey(ifm_empscode_to_stationid, empscode)
                    stationid = ifm_empscode_to_stationid[empscode]
                    ifm_replacemap[e.instancename] = stationid
                    push!(ifm_names, stationid)
                end
            end
        end
        ifm_elements = JulES.TuLiPa.getelements(JSON.parsefile(joinpath(sti_dataset, "ifm_elements.json")), sti_dataset)
    end

    if JulES.has_onlyagghydro(settings)
        startmagdict = Dict()
        detailedrescopl = Dict()
        return Dict("elements" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl, 
            "iprogtype" => iprogtype, "ifm_replacemap" => ifm_replacemap, "ifm_names" => ifm_names, "ifm_weights" => ifm_weights, "ifm_normfactors" => ifm_normfactors, "ifm_elements" => ifm_elements)
    else
        elements = vcat(exogen,detdseries,detdstructure,thermal,windsol,transm,cons,inflow,fuel,nuclear)
        startmagdict = JulES.JSON.parsefile(joinpath(sti_dataset1, "startmagdict.json"))
        detailedrescopl = JulES.JSON.parsefile(joinpath(sti_dataset, "magasin_elspot.json"))
        return Dict("elements" => elements, "elements_ppp" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl, 
            "iprogtype" => iprogtype, "ifm_replacemap" => ifm_replacemap, "ifm_names" => ifm_names, "ifm_weights" => ifm_weights, "ifm_normfactors" => ifm_normfactors, "ifm_elements" => ifm_elements)
    end
end

### Run JulES and keep the results

In [ ]:
dataset = getdataset(config, weatheryear)
input = JulES.DefaultJulESInput(config, dataset, datayear, weatheryear)
JulES.cleanup_jules(input)
data = JulES.run_serial(input)

### Code to show results
- We don't show any results for this demo

In [ ]:
powerbalancenames = data["areanames"]
prices = data["pricematrix"]
x1 = data["priceindex"]

hydronames = data["resnames"]
hydrolevels = data["resmatrix"]
x2 = data["resindex"]

batterynames = data["batnames"]
batterylevels = data["batmatrix"]
x2 = data["batindex"]

statenames = data["statenames"]
statematrix = data["statematrix"]
x3 = data["stateindex"]

supplyvalues = data["supplyvalues"]
supplynames = data["supplynames"]
supplybalancenames = data["supplybalancenames"]

demandvalues = data["demandvalues"]
demandnames = data["demandnames"]
demandbalancenames = data["demandbalancenames"];

# storagevalues = data["storagevalues"]
# storagenames = data["storagenames"]
# scenarionames = data["scenarionames"]
# shorts = data["shorts"]
# skipfactor = data["skipfactor"];

# prices_long = data["prices_long"]
# prices_med = data["prices_med"]
# prices_short = data["prices_short"]
# deltas_long = data["deltas_long"]
# deltas_med = data["deltas_med"]
# deltas_short = data["deltas_short"]
# balancenames_ppp = data["balancenames_ppp"]

In [ ]:
a = 0.5
b = -4
c = 10
# a = 1
# b = 0
# c = 5
# a = 0
# b = 0
# c = 1
numscen = 7
x = collect(-numscen+1:2:numscen-1)
y = a .* x .^ 2 .+ x .* b .+ c
display(y/sum(y)) # show chosen weights

In [ ]:
# Plot prices
idxwohub = findall(x -> !occursin("HUB", x), powerbalancenames) # remove hubs, not active in 2025 dataset
p = plot(x1, prices[:,idxwohub]*100, labels=reshape(powerbalancenames[idxwohub],1,length(powerbalancenames[idxwohub])), size=(800,500), title="Prices", ylabel="€/MWh")

# # Also plot ppp prices at given intervals (here only at first step)
# interval = 2
# step = 41
# # scens = reshape([string(i) for i in 1:size(prices_long, 3)], 1, size(prices_long, 3))
# scens = reshape([string(1) for i in 1:size(prices_long, 3)], 1, size(prices_long, 3))
# x4 = [x3[step]]
# for i in 1:(size(deltas_long, 2)-1)
#     t = x3[step] + Millisecond(deltas_long[interval, i])
#     push!(x4, t)
# end
# areaix = findfirst(==("PowerBalance_SORLAND"), balancenames_ppp)
# plot!(p, x4, transpose(prices_long[interval, areaix, :, :]*-100), labels=scens)

# scens = reshape([string(2) for i in 1:size(prices_long, 3)], 1, size(prices_long, 3))
# x5 = [x3[step]]
# for i in 1:(size(deltas_med, 2)-1)
#     t = x3[step] + Millisecond(deltas_med[interval, i])
#     push!(x5, t)
# end
# plot!(p, x5, transpose(prices_med[interval, areaix, :, :]*-100), labels=scens)

# scens = reshape([string(3) for i in 1:size(prices_long, 3)], 1, size(prices_long, 3))
# x6 = [x3[step]]
# for i in 1:(size(deltas_short, 2)-1)
#     t = x3[step] + Millisecond(deltas_short[interval, i])
#     push!(x6, t)
# end
# plot!(p, x6, transpose(prices_short[interval, areaix, :, :]*-100), labels=scens)
display(p)

# # Plot supplies and demands
# maxdemsup = isempty(supplyvalues) ? maximum(demandvalues) : (isempty(demandvalues) ? maximum(supplyvalues) : max(maximum(demandvalues), maximum(supplyvalues)))
# supplychart = plot(x1, supplyvalues,labels=reshape(supplynames,1,length(supplynames)),title="Supply", ylabel = "GWh/h", ylims=(0,maxdemsup))
# demandchart = plot(x1, demandvalues,labels=reshape(demandnames,1,length(demandnames)),title="Demand", ylabel = "GWh/h", ylims=(0,maxdemsup))
sumsupplyvalues = sum(supplyvalues,dims=2)
sumdemandvalues = sum(demandvalues,dims=2)
maxdemsup = isempty(sumsupplyvalues) ? maximum(sumdemandvalues) : (isempty(sumdemandvalues) ? maximum(sumsupplyvalues) : max(maximum(sumdemandvalues), maximum(sumsupplyvalues)))
supplychart = areaplot(x1,sumsupplyvalues,title="Supply", ylabel = "GWh/h")
demandchart = areaplot(x1,sumdemandvalues,title="Demand", ylabel = "GWh/h")
display(plot([supplychart,demandchart]...,layout=(1,2),size=(800,500)))
# display(plot(supplychart,size=(800,500)))

# Plot storages
# display(areaplot(x2, hydrolevels1,labels=reshape(hydronames,1,length(hydronames)),size=(800,500),title="Reservoir levels", ylabel = "TWh")) #
display(areaplot(x2, dropdims(sum(hydrolevels,dims=2),dims=2),labels="Total",size=(800,500),title="Reservoir levels", ylabel = "TWh")) #

# display(areaplot(x1, dropdims(sum(batterylevels,dims=2),dims=2),labels="Total",size=(800,500),title="Short term storage levels", ylabel = "GWh")) #

# Plot list of yearly mean production and demand for each supply/demand TODO: split demand/supply and transmission
meandemand = dropdims(mean(demandvalues,dims=1),dims=1)
meanproduction = dropdims(mean(supplyvalues,dims=1),dims=1)
supplydf = sort(DataFrame(Supplyname = supplynames, Yearly_supply_TWh = meanproduction*8.76),[:Yearly_supply_TWh], rev = true)
demanddf = sort(DataFrame(Demandname = demandnames, Yearly_demand_TWh = meandemand*8.76),[:Yearly_demand_TWh], rev = true)
supplydf[!,:ID] = collect(1:length(supplynames))
demanddf[!,:ID] = collect(1:length(demandnames))
joineddf = select!(outerjoin(supplydf,demanddf;on=:ID),Not(:ID))
pretty_table(joineddf, show_subheader = true)

# Check that total supply equals total demand
pretty_table(combine(joineddf, [:Yearly_supply_TWh, :Yearly_demand_TWh] .=> sum∘skipmissing), show_subheader = true)

# # Plot list of yearly income and cost for each supply/demand (only works if exogenprices are collected)
# supplyrev = copy(supplyvalues)
# for (i,supplybalancename) in enumerate(supplybalancenames)
#     idx = findfirst(isequal(supplybalancename), powerbalancenames)
#     supplyrev[:,i] .= supplyrev[:,i] .* prices[:,idx]
# end
# demandrev = copy(demandvalues)
# for (i,demandbalancename) in enumerate(demandbalancenames)
#     idx = findfirst(isequal(demandbalancename), powerbalancenames)
#     demandrev[:,i] .= demandrev[:,i] .* prices[:,idx]
# end
# meandemandrev = dropdims(mean(demandrev,dims=1),dims=1)
# meanproductionrev = dropdims(mean(supplyrev,dims=1),dims=1)
# supplyrevdf = sort(DataFrame(Supplyname = supplynames, Yearly_rev_mill€ = meanproductionrev*8.76),[:Yearly_rev_mill€], rev = true)
# demandrevdf = sort(DataFrame(Demandname = demandnames, Yearly_cost_mill€ = meandemandrev*8.76),[:Yearly_cost_mill€], rev = true)
# supplyrevdf[!,:ID] = collect(1:length(supplynames))
# demandrevdf[!,:ID] = collect(1:length(demandnames))
# joinedrevdf = select!(outerjoin(supplyrevdf,demandrevdf;on=:ID),Not(:ID))
# # pretty_table(joinedrevdf, show_subheader = true)

# # Sum revenues and cost
# pretty_table(combine(joinedrevdf, [:Yearly_rev_mill€, :Yearly_cost_mill€] .=> sum∘skipmissing), show_subheader = true)

# # Plot storagevalues for each reservoir and scenarios
# maxlongtermstorages = 30
# maxshorttermstorages = 30
# shortindex = x3
# medindex = x3[1:Int(skipfactor):end]
# numstoch = length(findall(sn -> occursin("min", sn), scenarionames))*2
# numop = length(findall(sn -> occursin("Operative", sn), scenarionames))
# numop_acc = numstoch + numop
# numstochend = length(findall(sn -> occursin("stochend", sn), scenarionames))
# numstochend_acc = numop_acc + numstochend
# numevpend = length(findall(sn -> occursin("evpend", sn), scenarionames))
# numevpend_acc = numstochend_acc + numevpend
# j = 0
# k = 0
# for (i, storagename) in enumerate(storagenames)
#     if shorts[i]
#         j += 1
#         j > maxshorttermstorages && continue
#         storagevalues_ = storagevalues[:,:,:]
#         index = shortindex
#     else
#         k += 1
#         k > maxlongtermstorages && continue
#         storagevalues_ = storagevalues[1:Int(skipfactor):end,:,:]
#         index = medindex
#     end
#     p = plot(index, storagevalues_[:,1:numstoch,i] * -100, size=(800,500), title="Storagevalues scenario and operative for " * storagename, labels=reshape(scenarionames[1:numstoch], 1, numstoch), ylabel="€/MWh")
#     plot!(p, index, storagevalues_[:,numstoch+1:numop_acc,i] * -100, labels=reshape(scenarionames[numstoch+1:numop_acc], 1, numop), linewidth=5)
#     plot!(p, index, storagevalues_[:,numop_acc+1:numstochend_acc,i] * -100, labels=reshape(scenarionames[numop_acc+1:numstochend_acc], 1, numstochend))
#     if numevpend > 0
#         plot!(p, index, storagevalues_[:,numstochend_acc+1:numevpend_acc,i] * -100, labels=reshape(scenarionames[numstochend_acc+1:numevpend_acc], 1, numevpend))
#     end
#     display(p)
# end